In [1]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
import numpy as np
import pickle
import json
import glob
import os

import matplotlib.pyplot as plt
%matplotlib inline

from enterprise.signals import signal_base
from enterprise.signals import gp_signals
from enterprise.pulsar import Pulsar

from enterprise_extensions import model_utils, blocks
from enterprise_extensions import model_orfs
from enterprise_extensions.frequentist import optimal_statistic as opt_stat

import h5py
import la_forge.core as co

In [2]:
#EPTA: Core and noise params 

In [3]:
c0 = co.Core(chaindir='/Users/physics/nanogw/data/chains_epta_dr2/')
noisefile = '/Users/physics/nanogw/data/epta_6psr_TN_noise.json'
with open(noisefile, 'r') as f:
    noisedict = json.load(f)

In [4]:
for p in c0.params:
    if p in noisedict:
        print("Exists:", p)
    else: 
        print('Does not exist:', p)

Exists: J0613-0200_dm_gp_gamma
Exists: J0613-0200_dm_gp_log10_A
Exists: J0613-0200_red_noise_gamma
Exists: J0613-0200_red_noise_log10_A
Exists: J1012+5307_dm_gp_gamma
Exists: J1012+5307_dm_gp_log10_A
Exists: J1012+5307_red_noise_gamma
Exists: J1012+5307_red_noise_log10_A
Exists: J1600-3053_dm_gp_gamma
Exists: J1600-3053_dm_gp_log10_A
Exists: J1600-3053_red_noise_gamma
Exists: J1600-3053_red_noise_log10_A
Exists: J1713+0747_dm_gp_gamma
Exists: J1713+0747_dm_gp_log10_A
Exists: J1713+0747_red_noise_gamma
Exists: J1713+0747_red_noise_log10_A
Exists: J1744-1134_dm_gp_gamma
Exists: J1744-1134_dm_gp_log10_A
Exists: J1744-1134_red_noise_gamma
Exists: J1744-1134_red_noise_log10_A
Exists: J1909-3744_dm_gp_gamma
Exists: J1909-3744_dm_gp_log10_A
Exists: J1909-3744_red_noise_gamma
Exists: J1909-3744_red_noise_log10_A
Does not exist: gw_log10_A
Does not exist: lnpost
Does not exist: lnlike
Does not exist: chain_accept
Does not exist: pt_chain_accept


In [5]:
#EPTA: Core and pta

In [6]:
import pickle
with open('/Users/physics/nanogw/data/epta_6psr_v0.pkl','rb') as fin:
    psrs=pickle.load(fin)

In [7]:
%%time
# (Note: It may take a few minutes to run this cell and may require at least ~4GB RAM)
# Initialize the optimal statistic object
# You can give it a list of pulsars and the noise dictionary, and it will create the pta object for you
# Alternatively, you can make the pta object yourself and give it to the OptimalStatistic object as an argument

# find the maximum time span to set GW frequency sampling
Tspan = model_utils.get_tspan(psrs)

# Here we build the signal model
# First we add the timing model
s = gp_signals.TimingModel()

# Then we add the white noise
# There are three types of white noise: EFAC, EQUAD, 
# We use different white noise parameters for every backend/receiver combination
# The white noise parameters are held constant
#inc_ecorr======FALSE because ppta
s += blocks.white_noise_block(vary=False, inc_ecorr=False, select='backend', tnequad=True)

#DMGP
dm = blocks.dm_noise_block(gp_kernel="diag", psd="powerlaw", components=100, Tspan=Tspan)

s+=dm

# Next comes the individual pulsar red noise
# We model the red noise as a Fourier series with 30 frequency components, 
# with a power-law PSD
s += blocks.red_noise_block(prior='log-uniform', Tspan=Tspan, components=30)

# Finally, we add the common red noise, which is modeled as a Fourier series with 5 frequency components
# The common red noise has a power-law PSD with spectral index of 4.33
s += blocks.common_red_noise_block(psd='powerlaw', prior='log-uniform', Tspan=Tspan, 
                                   components=5, gamma_val=4.33, name='gw')
models=[]
for p in psrs:
    if p.name not in ['J0437', 'J1713+0747_epta']:
        models.append(s(p))
    elif p.name=='J0437':
        models.append(s(p))
#         models.append((s+j0437dip)(p))
    elif p.name=='J1713+0747_epta':
        models.append(s(p))
#         models.append((s+j1713dips)(p))

# We set up the PTA object using the signal we defined above and the pulsars

pta = signal_base.PTA(models)

# We need to set the white noise parameters to the values in the noise dictionary
pta.set_default_params(noisedict)

CPU times: user 2.83 s, sys: 29.7 ms, total: 2.86 s
Wall time: 2.89 s


In [8]:
for p in c0.params:
    if p in pta.param_names:
        print("Exists:", p)
    else: 
        print('Does not exist:', p)

Exists: J0613-0200_dm_gp_gamma
Exists: J0613-0200_dm_gp_log10_A
Exists: J0613-0200_red_noise_gamma
Exists: J0613-0200_red_noise_log10_A
Exists: J1012+5307_dm_gp_gamma
Exists: J1012+5307_dm_gp_log10_A
Exists: J1012+5307_red_noise_gamma
Exists: J1012+5307_red_noise_log10_A
Exists: J1600-3053_dm_gp_gamma
Exists: J1600-3053_dm_gp_log10_A
Exists: J1600-3053_red_noise_gamma
Exists: J1600-3053_red_noise_log10_A
Exists: J1713+0747_dm_gp_gamma
Exists: J1713+0747_dm_gp_log10_A
Exists: J1713+0747_red_noise_gamma
Exists: J1713+0747_red_noise_log10_A
Exists: J1744-1134_dm_gp_gamma
Exists: J1744-1134_dm_gp_log10_A
Exists: J1744-1134_red_noise_gamma
Exists: J1744-1134_red_noise_log10_A
Exists: J1909-3744_dm_gp_gamma
Exists: J1909-3744_dm_gp_log10_A
Exists: J1909-3744_red_noise_gamma
Exists: J1909-3744_red_noise_log10_A
Exists: gw_log10_A
Does not exist: lnpost
Does not exist: lnlike
Does not exist: chain_accept
Does not exist: pt_chain_accept


In [9]:
#NANOGW: Core and noise params assume that ml_params has all params as core because 
#I am sure of where (if I ever had) my nanograv core is

In [10]:
datadir = './data'
if not os.path.isdir(datadir):
    datadir = '../data'
    
noisefile = datadir + '/channelized_12p5yr_v3_full_noisedict.json'

with open(noisefile, 'r') as f:
    noisedict = json.load(f)

with open(datadir + '/12p5yr_maxlike.json', 'r') as f:
    ml_params = json.load(f)

In [11]:
for p in ml_params.keys():
    if p in noisedict:
        print("Exists:", p)
    else: 
        print('Does not exist:', p)

Exists: B1855+09_red_noise_gamma
Exists: B1855+09_red_noise_log10_A
Exists: B1937+21_red_noise_gamma
Exists: B1937+21_red_noise_log10_A
Exists: B1953+29_red_noise_gamma
Exists: B1953+29_red_noise_log10_A
Exists: J0023+0923_red_noise_gamma
Exists: J0023+0923_red_noise_log10_A
Exists: J0030+0451_red_noise_gamma
Exists: J0030+0451_red_noise_log10_A
Exists: J0340+4130_red_noise_gamma
Exists: J0340+4130_red_noise_log10_A
Exists: J0613-0200_red_noise_gamma
Exists: J0613-0200_red_noise_log10_A
Exists: J0636+5128_red_noise_gamma
Exists: J0636+5128_red_noise_log10_A
Exists: J0645+5158_red_noise_gamma
Exists: J0645+5158_red_noise_log10_A
Exists: J0740+6620_red_noise_gamma
Exists: J0740+6620_red_noise_log10_A
Exists: J0931-1902_red_noise_gamma
Exists: J0931-1902_red_noise_log10_A
Exists: J1012+5307_red_noise_gamma
Exists: J1012+5307_red_noise_log10_A
Exists: J1024-0719_red_noise_gamma
Exists: J1024-0719_red_noise_log10_A
Exists: J1125+7819_red_noise_gamma
Exists: J1125+7819_red_noise_log10_A
Exis

In [12]:
#NANOGrav: Core and pta 
#Assume that ml_params has all params as core because I am sure of where 

In [13]:
noisedictcp=noisedict.copy()
for ky,val in noisedict.items():
    if 'equad'in ky and not 'tnequad' in ky:
        kynew=ky.replace('equad','tnequad')
        noisedictcp[kynew] = noisedict[ky]

In [14]:
import pickle
with open('/Users/physics/nanogw/data/ng_channelized_12yr_v3_partim_py3.pkl','rb') as fin:
    psrs=pickle.load(fin)

In [15]:
%%time
# (Note: It may take a few minutes to run this cell and may require at least ~4GB RAM)
# Initialize the optimal statistic object
# You can give it a list of pulsars and the noise dictionary, and it will create the pta object for you
# Alternatively, you can make the pta object yourself and give it to the OptimalStatistic object as an argument

# find the maximum time span to set GW frequency sampling
Tspan = model_utils.get_tspan(psrs)

# Here we build the signal model
# First we add the timing model
s = gp_signals.TimingModel()

# Then we add the white noise
# There are three types of white noise: EFAC, EQUAD, and ECORR --EQUAD==tnEquad
# We use different white noise parameters for every backend/receiver combination
# The white noise parameters are held constant
s += blocks.white_noise_block(vary=False, inc_ecorr=True, select='backend', tnequad=True)

# Next comes the individual pulsar red noise
# We model the red noise as a Fourier series with 30 frequency components, 
# with a power-law PSD
s += blocks.red_noise_block(prior='log-uniform', Tspan=Tspan, components=30)

# Finally, we add the common red noise, which is modeled as a Fourier series with 5 frequency components
# The common red noise has a power-law PSD with spectral index of 4.33
s += blocks.common_red_noise_block(psd='powerlaw', prior='log-uniform', Tspan=Tspan, 
                                   components=5, gamma_val=4.33, name='gw')

models=[]
for p in psrs:
    models.append(s(p))

# We set up the PTA object using the signal we defined above and the pulsars
pta = signal_base.PTA([s(p) for p in psrs])

# We need to set the white noise parameters to the values in the noise dictionary
pta.set_default_params(noisedictcp)


CPU times: user 27.2 s, sys: 1.23 s, total: 28.4 s
Wall time: 28.6 s


In [16]:
for p in ml_params.keys():
    if p in pta.param_names:
        print("Exists:", p)
    else: 
        print('Does not exist:', p)

Exists: B1855+09_red_noise_gamma
Exists: B1855+09_red_noise_log10_A
Exists: B1937+21_red_noise_gamma
Exists: B1937+21_red_noise_log10_A
Exists: B1953+29_red_noise_gamma
Exists: B1953+29_red_noise_log10_A
Exists: J0023+0923_red_noise_gamma
Exists: J0023+0923_red_noise_log10_A
Exists: J0030+0451_red_noise_gamma
Exists: J0030+0451_red_noise_log10_A
Exists: J0340+4130_red_noise_gamma
Exists: J0340+4130_red_noise_log10_A
Exists: J0613-0200_red_noise_gamma
Exists: J0613-0200_red_noise_log10_A
Exists: J0636+5128_red_noise_gamma
Exists: J0636+5128_red_noise_log10_A
Exists: J0645+5158_red_noise_gamma
Exists: J0645+5158_red_noise_log10_A
Exists: J0740+6620_red_noise_gamma
Exists: J0740+6620_red_noise_log10_A
Exists: J0931-1902_red_noise_gamma
Exists: J0931-1902_red_noise_log10_A
Exists: J1012+5307_red_noise_gamma
Exists: J1012+5307_red_noise_log10_A
Exists: J1024-0719_red_noise_gamma
Exists: J1024-0719_red_noise_log10_A
Exists: J1125+7819_red_noise_gamma
Exists: J1125+7819_red_noise_log10_A
Exis

In [17]:
#PTTA: Core and noise

In [18]:
noisefile = '/Users/physics/nanogw/data/ppta_dr2_TN_noise.json'

with open(noisefile, 'r') as f:
    noisedict = json.load(f)

In [19]:
c0 = co.Core(chaindir='/Users/physics/nanogw/data/chains_ppta_dr2/')

In [20]:
for p in c0.params:
    if p in noisedict:
        print("Exists:", p)
    else: 
        print('Does not exist:', p)

Does not exist: J0437-4715_band_noise_7_20CM_gamma
Does not exist: J0437-4715_band_noise_7_20CM_log10_A
Does not exist: J0437-4715_band_noise_by_B_4050CM_4050CM_gamma
Does not exist: J0437-4715_band_noise_by_B_4050CM_4050CM_log10_A
Does not exist: J0437-4715_chromatic_gp_gamma
Does not exist: J0437-4715_chromatic_gp_log10_A
Exists: J0437-4715_dm_gp_gamma
Exists: J0437-4715_dm_gp_log10_A
Does not exist: J0437-4715_dmexp_idx
Does not exist: J0437-4715_dmexp_log10_Amp
Does not exist: J0437-4715_dmexp_log10_tau
Does not exist: J0437-4715_dmexp_t0
Does not exist: J0437-4715_fd1_sys_CASPSR_40CM_CASPSR_40CM_slope
Does not exist: J0437-4715_fd1_sys_CPSR2_50CM_CPSR2_50CM_slope
Does not exist: J0437-4715_fd1_sys_PDFB1_10CM_PDFB1_10CM_slope
Does not exist: J0437-4715_fd1_sys_PDFB1_1433_PDFB1_1433_slope
Does not exist: J0437-4715_fd1_sys_PDFB_20CM_PDFB_20CM_slope
Does not exist: J0437-4715_fd1_sys_PDFB_40CM_PDFB_40CM_slope
Does not exist: J0437-4715_fd1_sys_WBCORR_10CM_WBCORR_10CM_slope
Exists: J0

In [38]:
for p in noisedict.keys():
    if p.endswith('_efac') or p.endswith('tnequad') :
        print("white noise")
    else:   
        if p in c0.params:
            print("Exists:", p)
        else: 
            print('Does not exist:', p)

Does not exist: J0437-4715_0_magnexp__idx
Does not exist: J0437-4715_0_magnexp__log10_Amp
Does not exist: J0437-4715_0_magnexp__log10_tau
Does not exist: J0437-4715_0_magnexp__t0
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
Exists: J0437-4715_dm_gp_gamma
Exists: J0437-4715_dm_gp_log10_A
Exists: J0437-4715_red_noise_gamma
Exists: J0437-4715_red_noise_log10_A
Exists: nmodel
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
Does not exist: J0613-0200_dm_gp_gamma
Does not exist: J0613-0200_dm_g

In [22]:
with open('/Users/physics/nanogw/data/ppta_dr2.pkl','rb') as fin:
    psrs=pickle.load(fin)

In [23]:
%%time
# (Note: It may take a few minutes to run this cell and may require at least ~4GB RAM)
# Initialize the optimal statistic object
# You can give it a list of pulsars and the noise dictionary, and it will create the pta object for you
# Alternatively, you can make the pta object yourself and give it to the OptimalStatistic object as an argument

# find the maximum time span to set GW frequency sampling
Tspan = model_utils.get_tspan(psrs)

# Here we build the signal model
# First we add the timing model
s = gp_signals.TimingModel()

# Then we add the white noise
# There are three types of white noise: EFAC, EQUAD, 
# We use different white noise parameters for every backend/receiver combination
# The white noise parameters are held constant
#inc_ecorr======FALSE because ppta
s += blocks.white_noise_block(vary=False, inc_ecorr=False, select='backend', tnequad=True)

#DMGP
dm = blocks.dm_noise_block(gp_kernel="diag", psd="powerlaw", components=100, Tspan=Tspan)

# Next comes the individual pulsar red noise
# We model the red noise as a Fourier series with 30 frequency components, 
# with a power-law PSD
s += blocks.red_noise_block(prior='log-uniform', Tspan=Tspan, components=30)

# Finally, we add the common red noise, which is modeled as a Fourier series with 5 frequency components
# The common red noise has a power-law PSD with spectral index of 4.33
s += blocks.common_red_noise_block(psd='powerlaw', prior='log-uniform', Tspan=Tspan, 
                                   components=5, gamma_val=4.33, name='gw')
models=[]
for p in psrs:
    if p.name not in ['J0437-4715_ppta', 'J1713+0747_ppta', 'J0613-0200_ppta','J1017-7156_ppta','J1045-4509_ppta']:
        models.append((s+dm)(p))
    elif p.name=='J0437-4715_ppta':
        models.append(s(p))
#         models.append((s+dm+j0437dip)(p))
    elif p.name=='J1713+0747_ppta':
        models.append(s(p))
#         models.append((s+dm+j1713dips)(p))
    elif p.name in [ 'J0613-0200_ppta','J1017-7156_ppta','J1045-4509_ppta']:
        models.append(s(p))
# We set up the PTA object using the signal we defined above and the pulsars

"""
import cloudpickle
with open('/Users/physics/nanogw/data/ppta_models.pkl','wb') as fout:
    cloudpickle.dump(models,fout)

"""
pta = signal_base.PTA(models)

# We need to set the white noise parameters to the values in the noise dictionary
pta.set_default_params(noisedict)

CPU times: user 11.7 s, sys: 479 ms, total: 12.2 s
Wall time: 12.3 s


In [24]:
for p in c0.params:
    if p in pta.param_names:
        print("Exists:", p)
    else: 
        print('Does not exist:', p)

Does not exist: J0437-4715_band_noise_7_20CM_gamma
Does not exist: J0437-4715_band_noise_7_20CM_log10_A
Does not exist: J0437-4715_band_noise_by_B_4050CM_4050CM_gamma
Does not exist: J0437-4715_band_noise_by_B_4050CM_4050CM_log10_A
Does not exist: J0437-4715_chromatic_gp_gamma
Does not exist: J0437-4715_chromatic_gp_log10_A
Exists: J0437-4715_dm_gp_gamma
Exists: J0437-4715_dm_gp_log10_A
Does not exist: J0437-4715_dmexp_idx
Does not exist: J0437-4715_dmexp_log10_Amp
Does not exist: J0437-4715_dmexp_log10_tau
Does not exist: J0437-4715_dmexp_t0
Does not exist: J0437-4715_fd1_sys_CASPSR_40CM_CASPSR_40CM_slope
Does not exist: J0437-4715_fd1_sys_CPSR2_50CM_CPSR2_50CM_slope
Does not exist: J0437-4715_fd1_sys_PDFB1_10CM_PDFB1_10CM_slope
Does not exist: J0437-4715_fd1_sys_PDFB1_1433_PDFB1_1433_slope
Does not exist: J0437-4715_fd1_sys_PDFB_20CM_PDFB_20CM_slope
Does not exist: J0437-4715_fd1_sys_PDFB_40CM_PDFB_40CM_slope
Does not exist: J0437-4715_fd1_sys_WBCORR_10CM_WBCORR_10CM_slope
Exists: J0

In [37]:
for p in noisedict.keys():
    if p.endswith('_efac') or p.endswith('tnequad') :
        print("white noise")
    else:
        if p in pta.param_names:
            print("Exists:", p)
        else: 
            print('Does not exist:', p)

Does not exist: J0437-4715_0_magnexp__idx
Does not exist: J0437-4715_0_magnexp__log10_Amp
Does not exist: J0437-4715_0_magnexp__log10_tau
Does not exist: J0437-4715_0_magnexp__t0
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
Exists: J0437-4715_dm_gp_gamma
Exists: J0437-4715_dm_gp_log10_A
Exists: J0437-4715_red_noise_gamma
Exists: J0437-4715_red_noise_log10_A
Does not exist: nmodel
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
white noise
Exists: J0613-0200_dm_gp_gamma
Exists: J0613-0200_dm_gp_log10_